In [93]:
import os

import requests
from IPython.display import HTML
import html as htmla
import csv
from tqdm.notebook import tqdm

from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient, DocumentIntelligenceAdministrationClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest


In [90]:
from dotenv import load_dotenv
load_dotenv("../../credentials.env")

True

In [88]:
endpoint = os.environ["INTELLIGENT_DOCUMENT_ENDPOINT"]
int_doc_key = os.environ["INTELLIGENT_DOCUMENT_KEY"]

credential = AzureKeyCredential(int_doc_key)
document_analysis_client = DocumentIntelligenceClient(endpoint, credential)
doc_int_admin_client=DocumentIntelligenceAdministrationClient(endpoint, credential) 
models = doc_int_admin_client.list_models()

print("We have models with the following ids: {}".format(
    ", ".join([m.model_id for m in models])
))


We have models with the following ids: prebuilt-contract, prebuilt-healthInsuranceCard.us, prebuilt-idDocument, prebuilt-invoice, prebuilt-layout, prebuilt-read, prebuilt-receipt, prebuilt-tax.us.1098, prebuilt-tax.us.1098E, prebuilt-tax.us.1098T, prebuilt-tax.us.1099A, prebuilt-tax.us.1099B, prebuilt-tax.us.1099C, prebuilt-tax.us.1099CAP, prebuilt-tax.us.1099DIV, prebuilt-tax.us.1099G, prebuilt-tax.us.1099H, prebuilt-tax.us.1099INT, prebuilt-tax.us.1099K, prebuilt-tax.us.1099LS, prebuilt-tax.us.1099LTC, prebuilt-tax.us.1099MISC, prebuilt-tax.us.1099NEC, prebuilt-tax.us.1099OID, prebuilt-tax.us.1099PATR, prebuilt-tax.us.1099Q, prebuilt-tax.us.1099QA, prebuilt-tax.us.1099R, prebuilt-tax.us.1099S, prebuilt-tax.us.1099SA, prebuilt-tax.us.1099SB, prebuilt-tax.us.w2


In [85]:
def read_pdf(file_path, fromurl=False,pages=None):
    if fromurl:
        poller = document_analysis_client.begin_analyze_document("prebuilt-layout", analyze_request=AnalyzeDocumentRequest(url_source=file_path),pages=pages,locale="en-US")
    else:
        with open(file_path, "rb") as f:
            #print(f)
            poller = document_analysis_client.begin_analyze_document("prebuilt-layout", analyze_request=f,pages=pages,content_type="application/octet-stream",locale="en-US")

    results = poller.result()
    return results

#file_path = "Datasheet_WorkshopPLUS.pdf" 
#file_path = "Cost_Estimate Lagos_NCC_Draft_Bridging11-25-2019.pdf"
#file_path = "1_Wells_Fargo_UBPR_LB.pdf"

doc_result = read_pdf("https://www.irs.gov/pub/irs-pdf/p1304.pdf",fromurl=True,pages="7")

In [44]:
def table_to_html(table_idx,table):
    resgions = table.bounding_regions
    table_html = "<b> Table Number : {},  Page : {}, Rows : {}, Columns : {} </b></br>".format(table_idx,resgions[0].page_number,table.row_count, table.column_count)
    table_html += "<table>"
    rows = [sorted([cell for cell in table.cells if cell.row_index == i], key=lambda cell: cell.column_index) for i in
            range(table.row_count)]
    for row_cells in rows:
        table_html += "<tr>"
        for cell in row_cells:
            tag = "th" if (cell.kind == "columnHeader" or cell.kind == "rowHeader") else "td"
            cell_spans = ""
            if cell.column_span and cell.column_span > 1: 
                cell_spans += f" colSpan={cell.column_span}"
            if cell.row_span and cell.row_span > 1: 
                cell_spans += f" rowSpan={cell.row_span}"
            content = htmla.escape(cell.content)
            tag_html = "<b>{}</b>".format(content) if (cell.kind == "columnHeader" or cell.kind == "rowHeader") else content
            table_html += f"<{tag}{cell_spans}>{tag_html}</{tag}>"
        table_html += "</tr>"
    table_html += "</table>"
    return table_html

In [79]:
#https://realpython.com/python-csv/

def table_to_csv(table_idx,table):
    print("\n\n\n")
    rows = [sorted([cell for cell in table.cells if cell.row_index == i], key=lambda cell: cell.column_index) for i in
            range(table.row_count)]
    with open('{}.csv'.format(table_idx), 'w',newline='') as out_file:
        writer = csv.writer(out_file,delimiter=',', quotechar='"',quoting=csv.QUOTE_MINIMAL)
        for row_cells in rows:
            #csv_row = ",".join([cell.content for cell in row_cells])
            csv_row = [cell.content for cell in row_cells]
            print(csv_row)
            writer.writerow(csv_row)
        

In [94]:
for table_idx, table in tqdm(enumerate(doc_result.tables)):
    htmltext = table_to_html(table_idx,table)
    table_to_csv(table_idx,table)

0it [00:00, ?it/s]





["'Item [1]", "'Current dollars", "'Percent change, 2019 to 2020"]
["'2016", "'2017", "'2018", "'2019", "'2020"]
["'", "'(1)", "'(2)", "'(3)", "'(4)", "'(5)", "'(6)"]
["'All returns", "'150,272,157", "'152,903,231", "'153,774,296", "'157,796,807", "'164,358,792", "'4.2"]
["'Electronically filed returns", "'131,618,295", "'135,016,593", "'137,645,234", "'142,162,470", "'151,569,468", "'6.6"]
["'Form 1040 returns", "'86,489,022", "'88,437,257", "'152,875,628", "'142,164,886", "'147,781,563", "'4.0"]
["'Electronically filed returns", "'73,560,523", "'75,826,047", "'137,310,575", "'128,737,582", "'137,029,512", "'6.4"]
["'Form 1040A returns", "'40,007,370", "'39,621,455", "'[2] 460,018", "'N/A", "'N/A", "'[3]"]
["'Electronically filed returns", "'36,945,040", "'36,842,523", "'[2] 163,420", "'N/A", "'N/A", "'[3]"]
["'Form 1040EZ returns", "'23,775,765", "'24,844,519", "'[2] 438,651", "'N/A", "'N/A", "'[3]"]
["'Electronically filed returns", "'21,112,732", "'22,348,023", "'[2] 171,238", 

In [95]:
for table_idx, table in tqdm(enumerate(doc_result.tables)):
    htmltext = table_to_html(table_idx,table)
    #print(htmltext)
    display(HTML(htmltext))

0it [00:00, ?it/s]

In [10]:
len(doc_result.tables)
for table_idx, table in enumerate(doc_result.tables):
    print(
        "Table # {} has {} rows and {} columns".format(
            table_idx, table.row_count, table.column_count
        )
    )
    for region in table.bounding_regions:
        print(
            "Table # {} location on page: {} is {}".format(
                table_idx,
                region.page_number,
                region.polygon,
            )
        )
    for cell in table.cells:
        print(
            "...Cell[{}][{}] has content '{}'".format(
                cell.row_index,
                cell.column_index,
                cell.content,
            )
        )
        for region in cell.bounding_regions:
            print(
                "...content on page {} is within bounding polygon '{}'".format(
                    region.page_number,
                    region.polygon,
                )
            )

print("----------------------------------------")

Table # 0 has 13 rows and 6 columns
Table # 0 location on page: 7 is [7.9966, 0.9764, 8.9046, 0.9792, 8.903, 1.6647, 7.9949, 1.6618]
...Cell[0][0] has content 'Date Estimate Prepared:
Explanation'
...content on page 7 is within bounding polygon '[7.8346, 0.9087, 9.0521, 0.9133, 9.0521, 0.9964, 7.8346, 0.9987]'
...Cell[1][0] has content 'For questions regarding this estimate, contact:'
...content on page 7 is within bounding polygon '[7.8346, 0.9987, 9.0521, 0.9964, 9.0521, 1.0449, 7.8346, 1.0473]'
...Cell[2][0] has content ''
...content on page 7 is within bounding polygon '[7.8346, 1.0473, 7.9397, 1.0473, 7.9397, 1.0842, 7.8346, 1.0842]'
...Cell[2][1] has content ''
...content on page 7 is within bounding polygon '[7.9397, 1.0473, 8.2595, 1.0473, 8.2595, 1.0819, 7.9397, 1.0842]'
...Cell[2][2] has content 'Bruce Buckin'
...content on page 7 is within bounding polygon '[8.2595, 1.0473, 8.5699, 1.0473, 8.5699, 1.0819, 8.2595, 1.0819]'
...Cell[2][3] has content ''
...content on page 7 is 